In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

1. Выведите количество пустых значений по каждой из колонок (название колонки, кол-во пустых значений)

In [6]:
pd.read_sql(
    """
    SELECT sum(case when crim is null then 1 else 0 end) as crim,
           sum(case when zn is null then 1 else 0 end) as zn,
           sum(case when indus is null then 1 else 0 end) as indus,
           sum(case when chas is null then 1 else 0 end) as chas,
           sum(case when nox is null then 1 else 0 end) as nox,
           sum(case when rm is null then 1 else 0 end) as rm,
           sum(case when age is null then 1 else 0 end) as age,
           sum(case when dis is null then 1 else 0 end) as dis,
           sum(case when rad is null then 1 else 0 end) as rad,
           sum(case when tax is null then 1 else 0 end) as tax,
           sum(case when ptratio is null then 1 else 0 end) as ptratio,
           sum(case when b is null then 1 else 0 end) as b,
           sum(case when lstat is null then 1 else 0 end) as lstat,
           sum(case when medv is null then 1 else 0 end) as medv
    FROM boston
    """,
    con,
)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Пустых значений, нет. УРА!

2. Выведите количество уникальных значений по каждой из колонок (название колонки, кол-во уникальных значений)

In [7]:
pd.read_sql(
    """
    with t AS (
    select 
         row_number() OVER (Partition BY crim ORDER BY crim) AS crim
         ,row_number() OVER (Partition BY zn ORDER BY zn) AS zn
         ,row_number() OVER (Partition BY indus ORDER BY indus) AS indus
         ,row_number() OVER (Partition BY chas ORDER BY chas) AS chas
         ,row_number() OVER (Partition BY nox ORDER BY nox) AS nox
         ,row_number() OVER (Partition BY rm ORDER BY rm) AS rm
         ,row_number() OVER (Partition BY age ORDER BY age) AS age
         ,row_number() OVER (Partition BY dis ORDER BY dis) AS dis
         ,row_number() OVER (Partition BY rad ORDER BY rad) AS rad
         ,row_number() OVER (Partition BY tax ORDER BY tax) AS tax
         ,row_number() OVER (Partition BY ptratio ORDER BY ptratio) AS ptratio
         ,row_number() OVER (Partition BY b ORDER BY b) AS b
         ,row_number() OVER (Partition BY lstat ORDER BY lstat) AS lstat
         ,row_number() OVER (Partition BY medv ORDER BY medv) AS medv
    FROM boston)
    select sum(case when crim == 1 then 1 else 0 end) as crim
          ,sum(case when zn == 1 then 1 else 0 end) as zn
          ,sum(case when indus == 1 then 1 else 0 end) as indus
          ,sum(case when chas == 1 then 1 else 0 end) as chas
          ,sum(case when nox == 1 then 1 else 0 end) as nox
          ,sum(case when rm == 1 then 1 else 0 end) as rm
          ,sum(case when age == 1 then 1 else 0 end) as age
          ,sum(case when dis == 1 then 1 else 0 end) as dis
          ,sum(case when rad == 1 then 1 else 0 end) as rad
          ,sum(case when tax == 1 then 1 else 0 end) as tax
          ,sum(case when ptratio == 1 then 1 else 0 end) as ptratio
          ,sum(case when b == 1 then 1 else 0 end) as b
          ,sum(case when lstat == 1 then 1 else 0 end) as lstat
          ,sum(case when medv == 1 then 1 else 0 end) as medv
    from t       
    """,
    con,
)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,504,26,76,2,81,446,356,412,9,66,46,357,455,229


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки). Напишите какой вывод можно сделать по данным в этих колонках

*Алексей, насколько я смог найти - в SQLlite нет встроенной функции поиска медианы. Для простоты я взял среднее между 253 и 254 записью. При необходимости можно написать функцию, но я решил не перегружать код*

In [8]:
pd.read_sql(
    """
    select avg(a.crim) - min (b.crim) AS diff_crim
          ,avg(a.zn) - min (b.zn) AS diff_zn
          ,avg(a.indus) - min (b.indus) AS diff_indus
          ,avg(a.chas) - min (b.chas) AS diff_chas
          ,avg(a.nox) - min (b.nox) AS diff_nox
          ,avg(a.rm) - min (b.rm) AS diff_rm
          ,avg(a.age) - min (b.age) AS diff_age
          ,avg(a.dis) - min (b.dis) AS diff_dis
          ,avg(a.rad) - min (b.rad) AS diff_rad
          ,avg(a.tax) - min (b.tax) AS diff_tax
          ,avg(a.ptratio) - min (b.ptratio) AS diff_ptratio
          ,avg(a.b) - min (b.b) AS diff_b
          ,avg(a.lstat) - min (b.lstat) AS diff_lstat
          ,avg(a.medv) - min (b.medv) AS diff_medv
    from (SELECT * 
    from boston ORDER BY crim LIMIT 2 OFFSET 253) a, boston b
    """,
    con,
)

,diff_crim,diff_zn,diff_indus,diff_chas,diff_nox,diff_rm,diff_age,diff_dis,diff_rad,diff_tax,diff_ptratio,diff_b,diff_lstat,diff_medv
0,0.2541,0.0,15.435,0.0,0.199,2.297,90.3,1.18155,3.0,183.5,7.2,393.885,12.725,12.8


Таким образом, получилось две колонки: zn и chas. Равенство медианы и минимального значения означает, что как минимум, половина всех значений в колонках равна минимальному значению, количество каждого из значений, отличных от минимального, намного меньше. Для колонки chas имеем вырожденный случай - всего два значения: 0 и 1, и опять же - количество 0 много больше количества 1.

В общем случае можно предположить, что распределение значений в таких колонках далеко от нормального и сильно смещено в сторону минимальных значений.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [9]:
pd.read_sql(
    """
    select 25 AS CNT, avg(a.rm) as avg_cheap, avg(b.rm) as avg_exp, avg(b.rm)-avg(a.rm) as DIFF from
    (SELECT 
      rm
    FROM boston ORDER BY MEDV
    limit 25) a,
    (SELECT 
      rm
    FROM boston ORDER BY MEDV DESC
    limit 25) b
    
    UNION
     
      select 50 AS CNT, avg(a.rm) as avg_cheap, avg(b.rm) as avg_exp, avg(b.rm)-avg(a.rm) as DIFF from
    (SELECT 
      rm
    FROM boston ORDER BY MEDV
    limit 50) a,
    (SELECT 
      rm
    FROM boston ORDER BY MEDV DESC
    limit 50) b 
    
    UNION
     
      select 100 AS CNT, avg(a.rm) as avg_cheap, avg(b.rm) as avg_exp, avg(b.rm)-avg(a.rm) as DIFF from
    (SELECT 
      rm
    FROM boston ORDER BY MEDV
    limit 100) a,
    (SELECT 
      rm
    FROM boston ORDER BY MEDV DESC
    limit 100) b 
    
    UNION
     
      select 200 AS CNT, avg(a.rm) as avg_cheap, avg(b.rm) as avg_exp, avg(b.rm)-avg(a.rm) as DIFF from
    (SELECT 
      rm
    FROM boston ORDER BY MEDV
    limit 200) a,
    (SELECT 
      rm
    FROM boston ORDER BY MEDV DESC
    limit 200) b  
    
    UNION
     
      select 300 AS CNT, avg(a.rm) as avg_cheap, avg(b.rm) as avg_exp, avg(b.rm)-avg(a.rm) as DIFF from
    (SELECT 
      rm
    FROM boston ORDER BY MEDV
    limit 300) a,
    (SELECT 
      rm
    FROM boston ORDER BY MEDV DESC
    limit 300) b 
    """,
    con,
)

,CNT,avg_cheap,avg_exp,DIFF
0,25,5.747840,7.637320,1.889480
1,50,5.753240,7.487060,1.733820
2,100,5.887120,7.198760,1.311640
3,200,5.911705,6.758680,0.846975
4,300,5.972227,6.539007,0.566780


Да, количество комнат и цена дома связанны прямой зависимостью. Для сравнения самых дешевых и самых дорогих домов разница в среднем составляет почти две конматы. При увеличении анализируемых интервалов в них включаются дома из среднего ценового диапазона и разница уменьшается. Для последенего случая средняя разница уменьшается до 0.5 комнаты, но при этом некоторые дома попадают одновременно как в верхний, так и в нижний интревал.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [10]:
pd.read_sql(
    """
    SELECT lstat, MEDV
          ,PERCENT_RANK() OVER(ORDER BY lstat) AS 'Percent_Rank'
    FROM boston
    ORDER BY MEDV DESC LIMIT 30
    """,
    con,
)

,LSTAT,MEDV,Percent_Rank
0,1.73,50.0,0.000000
1,1.92,50.0,0.001980
2,2.88,50.0,0.009901
3,2.96,50.0,0.013861
4,2.97,50.0,0.015842
5,3.16,50.0,0.027723
6,3.26,50.0,0.031683
7,3.32,50.0,0.033663
8,3.70,50.0,0.049505
9,3.73,50.0,0.051485


По выборке из 30 самых дорогих домов видно, что процент населения с более низким статусом в районе проживания низкий, а процентный ранг минимален. По мере уменьшения стоимости домов этот процент немного растет. 
**Вывод:** чем меньше процент населения с низким социальным статусом в районе проживания - тем дороже дом в таком районе.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [11]:
pd.read_sql(
    """
    SELECT DISTINCT (CASE when chas == 1 then 'Да' else 'Нет' end) as 'Граничит с рекой'
                 , avg(medv) OVER (Partition BY Chas) AS 'Средняя стоимость'
    FROM boston
    """,
    con,
)

,Граничит с рекой,Средняя стоимость
0,Нет,22.093843
1,Да,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки). Напишите какой вывод можно сделать по этим данным.

In [12]:
pd.read_sql(
    """
    with t1 as (
    SELECT DISTINCT (CASE when chas == 1 then 'Да' else 'Нет' end) as chas
                 , avg(crim) OVER (Partition BY Chas) as crim
                 , avg(zn)  OVER (Partition BY Chas) as zn
                 , avg(indus) OVER (Partition BY Chas) as indus
                 , avg(nox)  OVER (Partition BY Chas) as nox
                 , avg(rm)  OVER (Partition BY Chas) as rm
                 , avg(age)  OVER (Partition BY Chas) as age
                 , avg(dis)  OVER (Partition BY Chas) as dis
                 , avg(rad)  OVER (Partition BY Chas) as rad
                 , avg(tax)  OVER (Partition BY Chas) as tax
                 , avg(ptratio)  OVER (Partition BY Chas) as ptratio
                 , avg(b)  OVER (Partition BY Chas) as b
                 , avg(lstat)  OVER (Partition BY Chas) as lstat
                 , avg(medv) OVER (Partition BY Chas) AS medv
    FROM boston)
    select Name from (
    select distinct 'crim' as Name, 
    (select crim from t1 where chas =='Да') - (select crim from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'zn' as Name, 
    (select zn from t1 where chas =='Да') - (select zn from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'indus' as Name, 
    (select indus from t1 where chas =='Да') - (select indus from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'nox' as Name, 
    (select nox from t1 where chas =='Да') - (select nox from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'rm' as Name, 
    (select rm from t1 where chas =='Да') - (select rm from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'age' as Name, 
    (select age from t1 where chas =='Да') - (select age from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'dis' as Name, 
    (select dis from t1 where chas =='Да') - (select dis from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'rad' as Name, 
    (select rad from t1 where chas =='Да') - (select rad from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'tax' as Name, 
    (select tax from t1 where chas =='Да') - (select tax from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'ptratio' as Name, 
    (select ptratio from t1 where chas =='Да') - (select ptratio from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'b' as Name, 
    (select b from t1 where chas =='Да') - (select b from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'lstat' as Name, 
    (select lstat from t1 where chas =='Да') - (select lstat from t1 where chas =='Нет') as diff from t1
    union
    select distinct 'medv' as Name, 
    (select medv from t1 where chas =='Да') - (select medv from t1 where chas =='Нет') as diff from t1)
    t2 where t2.diff > 0
    """,
    con,
)

,Name
0,age
1,b
2,indus
3,medv
4,nox
5,rm


Участки, граничащие с рекой, в среднем находятся в более индустриальных районах, в которых выше концентрация оксида азота. В таких домах больше комнат и живут в них люди более старшего возраста. Так же у таких домов выше стоимость.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [13]:
pd.read_sql(
    """
    select * from (
    SELECT distinct
         NTILE(100) OVER(ORDER BY indus) AS 'ntile'
         ,indus, nox FROM boston) t1
    where ntile in (10,20,30,40,50,60,70,80,90,100)     
    """,
    con,
)

,ntile,indus,nox
0,10,2.93,0.4010
1,10,2.95,0.4280
2,10,2.97,0.4000
3,20,4.39,0.4420
4,20,4.49,0.4490
5,30,5.96,0.4990
6,30,6.06,0.4379
7,40,7.07,0.4690
8,40,7.38,0.4930
9,50,9.69,0.5850


В общем есть  зависимость: чем больше процент индустриальной застройки - тем больше концентрация оксида азота. Однако, при высоких процентилях (50 и выше) эта зависимость не такая явная, хотя в целом все равно концентрация на них выше, чем на низких процентилях. 

In [14]:
con.close()